In [1]:
from pyspark.sql import SparkSession, functions as F
import findspark
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)

from pyspark.sql.types import *
from pyspark.sql.functions import col,json_tuple,to_json,from_json,explode_outer

# /opt/manual/spark: this is SPARK_HOME path
findspark.init("/opt/manual/spark")

#spark = SparkSession.builder.config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.2.0").getOrCreate()

In [2]:
# spark & minio configuration on s3
spark = (SparkSession.builder
    .appName("Spark Minio Test")
    .config("spark.hadoop.fs.s3a.endpoint", "http://172.18.0.2:9000")
    .config("spark.hadoop.fs.s3a.access.key", 'root')
    .config("spark.hadoop.fs.s3a.secret.key", 'root12345')
    .config("spark.hadoop.fs.s3a.path.style.access", True)
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.2.0")
    .getOrCreate())

:: loading settings :: url = jar:file:/opt/manual/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c48a75aa-c213-458b-a3eb-1883c970255c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
:: resolution report :: resolve 319ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------------

In [5]:
credits =(spark.read.format("parquet")
     .option("header", True)
     .option("inferSchema",True)
     .load("s3a://tmdb-bronze/credits/credits_part_20221106-105003.parquet"))
credits.show(10,truncate=False)

credits.printSchema()

+--------+----------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
credits.limit(5).toPandas()

/home/train/venvairspark/lib/python3.8/site-packages/pyspark/sql/pandas/conversion.py:178: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [7]:
credits.count()

500

In [7]:
struct_cast = StructType([StructField('cast_id',IntegerType(),True),
                     StructField('character',StringType(),True),
                     StructField('credit_id',StringType(),True),
                     StructField('gender',IntegerType(),True),
                    StructField('id',IntegerType(),True),
                      StructField('name',StringType(),True),
                    StructField('order',IntegerType(),True)])

In [8]:
struct_crew= StructType([StructField('credit_id',StringType(),True),
                     StructField('department',StringType(),True),
                     StructField('gender',IntegerType(),True),
                     StructField('id',IntegerType(),True),
                    StructField('job',StringType(),True),
                      StructField('name',StringType(),True)])

In [9]:
credits_sch = credits.withColumn('cast',F.from_json(F.col('cast'),ArrayType(struct_cast))).withColumn('crew',F.from_json(F.col('crew'),ArrayType(struct_crew)))

In [10]:
credits_sch.printSchema()


root
 |-- movie_id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- cast: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cast_id: integer (nullable = true)
 |    |    |-- character: string (nullable = true)
 |    |    |-- credit_id: string (nullable = true)
 |    |    |-- gender: integer (nullable = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- order: integer (nullable = true)
 |-- crew: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- credit_id: string (nullable = true)
 |    |    |-- department: string (nullable = true)
 |    |    |-- gender: integer (nullable = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- job: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |-- event_time: timestamp (nullable = true)



# 2.1. Cast

In [65]:
cast_exp = credits_sch.select('movie_id','title',F.explode_outer('cast').alias('cast'))
cast_cols = ["cast_id","character","credit_id","gender","id","name","order"]
for col in cast_cols:
    cast_exp = cast_exp.withColumn(col, F.col("cast").getItem(col))
credits_cast = cast_exp.drop("cast").drop("order").na.fill(value=0000000000,subset=["credit_id"])

In [ ]:
# Cast table
# +--------+------+-------+-------------------+------------------------+------+-----+----------------+
# |movie_id|title |cast_id|character          |credit_id               |gender|id   |name            |
# +--------+------+-------+-------------------+------------------------+------+-----+----------------+
# |19995   |Avatar|242    |Jake Sully         |5602a8a7c3a3685532001c9a|2     |65731|Sam Worthington |
# |19995   |Avatar|3      |Neytiri            |52fe48009251416c750ac9cb|1     |8691 |Zoe Saldana     |
# |19995   |Avatar|25     |Dr. Grace Augustine|52fe48009251416c750aca39|1     |10205|Sigourney Weaver|
# |19995   |Avatar|4      |Col. Quaritch      |52fe48009251416c750ac9cf|2     |32747|Stephen Lang    |
# +--------+------+-------+-------------------+------------------------+------+-----+----------------+

# 2.2. crew

In [69]:
crew_exp = credits_sch.select('movie_id','title',F.explode_outer('crew').alias('crew'))

In [70]:
crew_cols = ["credit_id","department","gender","id","job","name"]

In [75]:
for col in crew_cols:
    crew_exp = crew_exp.withColumn(col, F.col("crew").getItem(col))
crew_exp = crew_exp.drop("crew").na.fill(value=0000000000,subset=["credit_id"])

In [76]:
crew_exp.limit(5).toPandas()

,movie_id,title,credit_id,department,gender,id,job,name
0,19995,Avatar,52fe48009251416c750aca23,Editing,0,1721,Editor,Stephen E. Rivkin
1,19995,Avatar,539c47ecc3a36810e3001f87,Art,2,496,Production Design,Rick Carter
2,19995,Avatar,54491c89c3a3680fb4001cf7,Sound,0,900,Sound Designer,Christopher Boyes
3,19995,Avatar,54491cb70e0a267480001bd0,Sound,0,900,Supervising Sound Editor,Christopher Boyes
4,19995,Avatar,539c4a4cc3a36810c9002101,Production,1,1262,Casting,Mali Finn


In [ ]:
# +--------+------+------------------------+----------+------+----+------------------------+-----------------+
# |movie_id|title |credit_id               |department|gender|id  |job                     |name             |
# +--------+------+------------------------+----------+------+----+------------------------+-----------------+
# |19995   |Avatar|52fe48009251416c750aca23|Editing   |0     |1721|Editor                  |Stephen E. Rivkin|
# |19995   |Avatar|539c47ecc3a36810e3001f87|Art       |2     |496 |Production Design       |Rick Carter      |
# |19995   |Avatar|54491c89c3a3680fb4001cf7|Sound     |0     |900 |Sound Designer          |Christopher Boyes|
# |19995   |Avatar|54491cb70e0a267480001bd0|Sound     |0     |900 |Supervising Sound Editor|Christopher Boyes|
# +--------+------+------------------------+----------+------+----+------------------------+-----------------+

# 2.3 Movies

In [77]:
movies =(spark.read.format("parquet")
     .option("header", True)
     .option("inferSchema",True)
     .load("s3a://tmdb-bronze/movies/movies_part_20221023-222937.parquet"))


+---------+--------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------+------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [78]:
movies.limit(5).toPandas()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,event_time
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 9685, ""name"": ""futuristic""}, {""id"": 9840, ""name"": ""romance""}, {""id"": 9882, ""name"": ""space""}, {""id"": 9951, ""name"": ""alien""}, {""id"": 10148, ""name"": ""tribe""}, {""id"": 10158, ""name"": ""alien planet""}, {""id"": 10987, ""name"": ""cgi""}, {""id"": 11399, ""name"": ""marine""}, {""id"": 13065, ""name"": ""soldier""}, {""id"": 14643, ""name"": ""battle""}, {""id"": 14720, ""name"": ""love affair""}, {""id"": 165431, ""name"": ""anti war""}, {""id"": 193554, ""name"": ""power relations""}, {""id"": 206690, ""name"": ""mind and soul""}, {""id"": 209714, ""name"": ""3d""}]",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporation"", ""id"": 306}, {""name"": ""Dune Entertainment"", ""id"": 444}, {""name"": ""Lightstorm Entertainment"", ""id"": 574}]",...,2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800,2022-10-24 00:28:46.761685
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic island""}, {""id"": 1319, ""name"": ""east india trading company""}, {""id"": 2038, ""name"": ""love of one's life""}, {""id"": 2052, ""name"": ""traitor""}, {""id"": 2580, ""name"": ""shipwreck""}, {""id"": 2660, ""name"": ""strong woman""}, {""id"": 3799, ""name"": ""ship""}, {""id"": 5740, ""name"": ""alliance""}, {""id"": 5941, ""name"": ""calypso""}, {""id"": 6155, ""name"": ""afterlife""}, {""id"": 6211, ""name"": ""fighter""}, {""id"": 12988, ""name"": ""pirate""}, {""id"": 157186, ""name"": ""swashbuckler""}, {""id"": 179430, ""name"": ""aftercreditsstinger""}]",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Films"", ""id"": 130}, {""name"": ""Second Mate Productions"", ""id"": 19936}]",...,2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,2022-10-24 00:28:46.862801
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret agent""}, {""id"": 9663, ""name"": ""sequel""}, {""id"": 14555, ""name"": ""mi6""}, {""id"": 156095, ""name"": ""british secret service""}, {""id"": 158431, ""name"": ""united kingdom""}]",en,Spectre,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While

In [79]:
movies.printSchema()

root
 |-- budget: long (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: long (nullable = true)
 |-- keywords: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- runtime: double (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: long (nullable = true)
 |-- event_time: timestamp (nullable = true)



In [113]:
movies_selected = movies.withColumnRenamed("id","movie_id").select("movie_id","title","budget","homepage","original_language","original_title","overview","popularity","release_date","revenue","runtime","status","tagline","vote_average","vote_count")

In [114]:
movies_selected.limit(5).toPandas()

,movie_id,title,budget,homepage,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,vote_average,vote_count
0,19995,Avatar,237000000,http://www.avatarmovie.com/,en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.437577,2009-12-10,2787965087,162.0,Released,Enter the World of Pandora.,7.2,11800
1,285,Pirates of the Caribbean: At World's End,300000000,http://disney.go.com/disneypictures/pirates/,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.082615,2007-05-19,961000000,169.0,Released,"At the end of the world, the adventure begins.",6.9,4500
2,206647,Spectre,245000000,http://www.sonypictures.com/movies/spectre/,en,Spectre,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.",107.376788,2015-10-26,880674609,148.0,Released,A Plan No One Escapes,6.3,4466
3,49026,The Dark Knight Rises,250000000,http://www.thedarkknightrises.com/,en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.",112.312950,2012-07-16,1084939099,165.0,Released,The Legend Ends,7.6,9106
4,49529,John Carter,260000000,http://movies.disney.com/john-carter,en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands.",43.926995,2012-03-07,284139100,132.0,Released,"Lost in our world, found in another.",6.1,2124


In [115]:
movies_selected.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- title: string (nullable = true)
 |-- budget: long (nullable = true)
 |-- homepage: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- runtime: double (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: long (nullable = true)



In [121]:
movies_sch = (movies_selected.withColumn("movie_id",F.col("movie_id").cast(StringType()))
    .withColumn("title",F.col("title").cast(StringType()))
    .withColumn("budget",F.col("budget").cast(DoubleType()))
    .withColumn("homepage",F.col("homepage").cast(StringType()))
    .withColumn("original_language",F.col("original_language").cast(StringType()))
 .withColumn("original_title",F.col("original_title").cast(StringType()))
 .withColumn("overview",F.col("overview").cast(StringType()))
 .withColumn("popularity",F.col("popularity").cast(FloatType()))
 .withColumn("release_date", F.to_date(F.col("release_date"),"yyyy-M-d"))
 .withColumn("revenue",F.col("revenue").cast(DoubleType()))
 .withColumn("runtime",F.col("runtime").cast(IntegerType()))
 .withColumn("status",F.col("status").cast(StringType()))
 .withColumn("tagline",F.col("tagline").cast(StringType()))
 .withColumn("vote_average",F.col("vote_average").cast(FloatType()))
 .withColumn("vote_count",F.col("vote_count").cast(IntegerType()))
 )

In [122]:
movies_sch.limit(5).toPandas()

,movie_id,title,budget,homepage,original_language,original_title,overview,popularity,release_date,revenue,runtime,status,tagline,vote_average,vote_count
0,19995,Avatar,237000000.0,http://www.avatarmovie.com/,en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.",150.437576,2009-12-10,2.787965e+09,162,Released,Enter the World of Pandora.,7.2,11800
1,285,Pirates of the Caribbean: At World's End,300000000.0,http://disney.go.com/disneypictures/pirates/,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.",139.082611,2007-05-19,9.610000e+08,169,Released,"At the end of the world, the adventure begins.",6.9,4500
2,206647,Spectre,245000000.0,http://www.sonypictures.com/movies/spectre/,en,Spectre,"A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.",107.376785,2015-10-26,8.806746e+08,148,Released,A Plan No One Escapes,6.3,4466
3,49026,The Dark Knight Rises,250000000.0,http://www.thedarkknightrises.com/,en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encounters the mysterious Selina Kyle and the villainous Bane, a new terrorist leader who overwhelms Gotham's finest. The Dark Knight resurfaces to protect a city that has branded him an enemy.",112.312950,2012-07-16,1.084939e+09,165,Released,The Legend Ends,7.6,9106
4,49529,John Carter,260000000.0,http://movies.disney.com/john-carter,en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably transported to the mysterious and exotic planet of Barsoom (Mars) and reluctantly becomes embroiled in an epic conflict. It's a world on the brink of collapse, and Carter rediscovers his humanity when he realizes the survival of Barsoom and its people rests in his hands.",43.926994,2012-03-07,2.841391e+08,132,Released,"Lost in our world, found in another.",6.1,2124


In [123]:
movies_sch.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- budget: double (nullable = true)
 |-- homepage: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: float (nullable = true)
 |-- release_date: date (nullable = true)
 |-- revenue: double (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- vote_average: float (nullable = true)
 |-- vote_count: integer (nullable = true)



# 2.4. genres

In [169]:
movies_genres = movies.withColumnRenamed("id","movie_id").select("movie_id","genres")
movies_genres.limit(5).toPandas()

,movie_id,genres
0,19995,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 878, ""name"": ""Science Fiction""}]"
1,285,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""name"": ""Action""}]"
2,206647,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""name"": ""Crime""}]"
3,49026,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name"": ""Drama""}, {""id"": 53, ""name"": ""Thriller""}]"
4,49529,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ""name"": ""Science Fiction""}]"


In [170]:
struct_genres = StructType([StructField('id',IntegerType(),True),
                     StructField('name',StringType(),True)])

In [171]:
genres_sch = movies_genres.withColumn('movie_id',F.col("movie_id").cast(StringType())).withColumn('genres',F.from_json(F.col('genres'),ArrayType(struct_genres)))

In [172]:
genres_sch.limit(5).toPandas()

,movie_id,genres
0,19995,"[(28, Action), (12, Adventure), (14, Fantasy), (878, Science Fiction)]"
1,285,"[(12, Adventure), (14, Fantasy), (28, Action)]"
2,206647,"[(28, Action), (12, Adventure), (80, Crime)]"
3,49026,"[(28, Action), (80, Crime), (18, Drama), (53, Thriller)]"
4,49529,"[(28, Action), (12, Adventure), (878, Science Fiction)]"


In [173]:
genres_sch.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)



In [174]:
genres_exp = genres_sch.select('movie_id',F.explode_outer('genres').alias('genres'))

In [175]:
genres_exp.limit(5).toPandas()

,movie_id,genres
0,19995,"(28, Action)"
1,19995,"(12, Adventure)"
2,19995,"(14, Fantasy)"
3,19995,"(878, Science Fiction)"
4,285,"(12, Adventure)"


In [178]:
genres_cols = ["id","name"]
for col in genres_cols:
    genres_exp = genres_exp.withColumn(col, F.col("genres").getItem(col))

In [179]:
genres_exp.limit(5).toPandas()

,movie_id,genres,id,name
0,19995,"(28, Action)",28,Action
1,19995,"(12, Adventure)",12,Adventure
2,19995,"(14, Fantasy)",14,Fantasy
3,19995,"(878, Science Fiction)",878,Science Fiction
4,285,"(12, Adventure)",12,Adventure


In [180]:
genres = genres_exp.drop("genres").na.fill(value=-9999, subset=["id"])
genres.limit(5).toPandas()

,movie_id,id,name
0,19995,28,Action
1,19995,12,Adventure
2,19995,14,Fantasy
3,19995,878,Science Fiction
4,285,12,Adventure


In [181]:
#genres.sort("id").show(truncate=False)

+--------+---+---------+
|movie_id|id |name     |
+--------+---+---------+
|49521   |12 |Adventure|
|37724   |12 |Adventure|
|2454    |12 |Adventure|
|135397  |12 |Adventure|
|24428   |12 |Adventure|
|206647  |12 |Adventure|
|1865    |12 |Adventure|
|559     |12 |Adventure|
|122917  |12 |Adventure|
|767     |12 |Adventure|
|1930    |12 |Adventure|
|1452    |12 |Adventure|
|20662   |12 |Adventure|
|58      |12 |Adventure|
|57158   |12 |Adventure|
|19995   |12 |Adventure|
|2268    |12 |Adventure|
|49529   |12 |Adventure|
|254     |12 |Adventure|
|209112  |12 |Adventure|
+--------+---+---------+
only showing top 20 rows



# 2.5 Keywords

In [192]:
movies_keywords = movies.withColumnRenamed("id","movie_id").select("movie_id","keywords")
movies_keywords.limit(5).toPandas()

,movie_id,keywords
0,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""space war""}, {""id"": 3388, ""name"": ""space colony""}, {""id"": 3679, ""name"": ""society""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 9685, ""name"": ""futuristic""}, {""id"": 9840, ""name"": ""romance""}, {""id"": 9882, ""name"": ""space""}, {""id"": 9951, ""name"": ""alien""}, {""id"": 10148, ""name"": ""tribe""}, {""id"": 10158, ""name"": ""alien planet""}, {""id"": 10987, ""name"": ""cgi""}, {""id"": 11399, ""name"": ""marine""}, {""id"": 13065, ""name"": ""soldier""}, {""id"": 14643, ""name"": ""battle""}, {""id"": 14720, ""name"": ""love affair""}, {""id"": 165431, ""name"": ""anti war""}, {""id"": 193554, ""name"": ""power relations""}, {""id"": 206690, ""name"": ""mind and soul""}, {""id"": 209714, ""name"": ""3d""}]"
1,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911, ""name"": ""exotic island""}, {""id"": 1319, ""name"": ""east india trading company""}, {""id"": 2038, ""name"": ""love of one's life""}, {""id"": 2052, ""name"": ""traitor""}, {""id"": 2580, ""name"": ""shipwreck""}, {""id"": 2660, ""name"": ""strong woman""}, {""id"": 3799, ""name"": ""ship""}, {""id"": 5740, ""name"": ""alliance""}, {""id"": 5941, ""name"": ""calypso""}, {""id"": 6155, ""name"": ""afterlife""}, {""id"": 6211, ""name"": ""fighter""}, {""id"": 12988, ""name"": ""pirate""}, {""id"": 157186, ""name"": ""swashbuckler""}, {""id"": 179430, ""name"": ""aftercreditsstinger""}]"
2,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4289, ""name"": ""secret agent""}, {""id"": 9663, ""name"": ""sequel""}, {""id"": 14555, ""name"": ""mi6""}, {""id"": 156095, ""name"": ""british secret service""}, {""id"": 158431, ""name"": ""united kingdom""}]"
3,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""id"": 949, ""name"": ""terrorist""}, {""id"": 1308, ""name"": ""secret identity""}, {""id"": 1437, ""name"": ""burglar""}, {""id"": 3051, ""name"": ""hostage drama""}, {""id"": 3562, ""name"": ""time bomb""}, {""id"": 6969, ""name"": ""gotham city""}, {""id"": 7002, ""name"": ""vigilante""}, {""id"": 9665, ""name"": ""cover-up""}, {""id"": 9715, ""name"": ""superhero""}, {""id"": 9990, ""name"": ""villainess""}, {""id"": 10044, ""name"": ""tragic hero""}, {""id"": 13015, ""name"": ""terrorism""}, {""id"": 14796, ""name"": ""destruction""}, {""id"": 18933, ""name"": ""catwoman""}, {""id"": 156082, ""name"": ""cat burglar""}, {""id"": 156395, ""name"": ""imax""}, {""id"": 173272, ""name"": ""flood""}, {""id"": 179093, ""name"": ""criminal underworld""}, {""id"": 230775, ""name"": ""batman""}]"
4,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": 1456, ""name"": ""medallion""}, {""id"": 3801, ""name"": ""space travel""}, {""id"": 7376, ""name"": ""princess""}, {""id"": 9951, ""name"": ""alien""}, {""id"": 10028, ""name"": ""steampunk""}, {""id"": 10539, ""name"": ""martian""}, {""id"": 10685, ""name"": ""escape""}, {""id"": 161511, ""name"": ""edgar rice burroughs""}, {""id"": 163252, ""name"": ""alien race""}, {""id"": 179102, ""name"": ""superhuman strength""}, {""id"": 190320, ""name"": ""mars civilization""}, {""id"": 195446, ""name"": ""sword and planet""}, {""id"": 207928, ""name"": ""19th century""}, {""id"": 209714, ""name"": ""3d""}]"


In [194]:
struct_keywords = StructType([StructField('id', IntegerType(), True),StructField('name', StringType(), True)])
keyword_sch = movies_keywords.withColumn('movie_id', F.col("movie_id").cast(StringType())).withColumn('keywords', F.from_json(F.col('keywords'),ArrayType(struct_keywords)))
keyword_sch.limit(5).toPandas()

,movie_id,keywords
0,19995,"[(1463, culture clash), (2964, future), (3386, space war), (3388, space colony), (3679, society), (3801, space travel), (9685, futuristic), (9840, romance), (9882, space), (9951, alien), (10148, tribe), (10158, alien planet), (10987, cgi), (11399, marine), (13065, soldier), (14643, battle), (14720, love affair), (165431, anti war), (193554, power relations), (206690, mind and soul), (209714, 3d)]"
1,285,"[(270, ocean), (726, drug abuse), (911, exotic island), (1319, east india trading company), (2038, love of one's life), (2052, traitor), (2580, shipwreck), (2660, strong woman), (3799, ship), (5740, alliance), (5941, calypso), (6155, afterlife), (6211, fighter), (12988, pirate), (157186, swashbuckler), (179430, aftercreditsstinger)]"
2,206647,"[(470, spy), (818, based on novel), (4289, secret agent), (9663, sequel), (14555, mi6), (156095, british secret service), (158431, united kingdom)]"
3,49026,"[(849, dc comics), (853, crime fighter), (949, terrorist), (1308, secret identity), (1437, burglar), (3051, hostage drama), (3562, time bomb), (6969, gotham city), (7002, vigilante), (9665, cover-up), (9715, superhero), (9990, villainess), (10044, tragic hero), (13015, terrorism), (14796, destruction), (18933, catwoman), (156082, cat burglar), (156395, imax), (173272, flood), (179093, criminal underworld), (230775, batman)]"
4,49529,"[(818, based on novel), (839, mars), (1456, medallion), (3801, space travel), (7376, princess), (9951, alien), (10028, steampunk), (10539, martian), (10685, escape), (161511, edgar rice burroughs), (163252, alien race), (179102, superhuman strength), (190320, mars civilization), (195446, sword and planet), (207928, 19th century), (209714, 3d)]"


In [195]:
keyword_sch.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- id: integer (nullable = true)
 |    |    |-- name: string (nullable = true)



In [199]:
keyword_exp = keyword_sch.select('movie_id', F.explode_outer('keywords').alias('keywords'))
keyword_cols = ["id", "name"]
for col in keyword_cols:
    keyword_exp = keyword_exp.withColumn(col, F.col("keywords").getItem(col))
keyword_exp.limit(5).toPandas()
keywords = keyword_exp.drop("keywords").na.fill(value=-9999, subset=["id"])
keywords.limit(5).toPandas()

,movie_id,id,name
0,19995,1463,culture clash
1,19995,2964,future
2,19995,3386,space war
3,19995,3388,space colony
4,19995,3679,society


In [200]:
#keywords.sort("id").show(truncate=False)

+--------+-----+----------------+
|movie_id|id   |name            |
+--------+-----+----------------+
|37786   |-9999|null            |
|1735    |-9999|null            |
|49852   |-9999|null            |
|79698   |-9999|null            |
|109418  |-9999|null            |
|453     |30   |individual      |
|12193   |65   |holiday         |
|8871    |65   |holiday         |
|364     |65   |holiday         |
|17979   |65   |holiday         |
|1581    |65   |holiday         |
|51052   |65   |holiday         |
|604     |83   |saving the world|
|411     |83   |saving the world|
|217     |83   |saving the world|
|49521   |83   |saving the world|
|87101   |83   |saving the world|
|1452    |83   |saving the world|
|296     |83   |saving the world|
|6479    |83   |saving the world|
+--------+-----+----------------+
only showing top 20 rows



# 2.6 production_companies

In [201]:
movie_pro_comp = movies.withColumnRenamed("id", "movie_id").select("movie_id", "production_companies")
struct_pro_comp = StructType([StructField('id', IntegerType(), True), StructField('name', StringType(), True)])
pro_comp_sch = movie_pro_comp.withColumn('movie_id', F.col("movie_id").cast(StringType())).withColumn('production_companies',F.from_json(F.col('production_companies'),ArrayType(struct_pro_comp)))

companies_exp = pro_comp_sch.select('movie_id', F.explode_outer('production_companies').alias('production_companies'))
companies_cols = ["id", "name"]
for col in companies_cols:
    companies_exp = companies_exp.withColumn(col, F.col("production_companies").getItem(col))

companies = companies_exp.drop("production_companies").na.fill(value=-9999, subset=["id"])

#companies.sort("id").show(truncate=False)

In [202]:
companies.limit(5).toPandas()

,movie_id,id,name
0,19995,289,Ingenious Film Partners
1,19995,306,Twentieth Century Fox Film Corporation
2,19995,444,Dune Entertainment
3,19995,574,Lightstorm Entertainment
4,285,2,Walt Disney Pictures


In [203]:
companies.sort("id").show(truncate=False)

+--------+---+--------------------+
|movie_id|id |name                |
+--------+---+--------------------+
|1894    |1  |Lucasfilm           |
|1893    |1  |Lucasfilm           |
|217     |1  |Lucasfilm           |
|1895    |1  |Lucasfilm           |
|158852  |2  |Walt Disney Pictures|
|38757   |2  |Walt Disney Pictures|
|49529   |2  |Walt Disney Pictures|
|285     |2  |Walt Disney Pictures|
|57201   |2  |Walt Disney Pictures|
|62211   |2  |Walt Disney Pictures|
|9543    |2  |Walt Disney Pictures|
|102651  |2  |Walt Disney Pictures|
|58      |2  |Walt Disney Pictures|
|12155   |2  |Walt Disney Pictures|
|105864  |2  |Walt Disney Pictures|
|68728   |2  |Walt Disney Pictures|
|20526   |2  |Walt Disney Pictures|
|49013   |2  |Walt Disney Pictures|
|10193   |2  |Walt Disney Pictures|
|1865    |2  |Walt Disney Pictures|
+--------+---+--------------------+
only showing top 20 rows



# 2.7. production_countries

In [206]:
movie_countries = movies.withColumnRenamed("id", "movie_id").select("movie_id", "production_countries")
struct_countries = StructType([StructField('iso_3166_1', StringType(), True), StructField('name', StringType(), True)])
countries_sch = movie_countries.withColumn('movie_id', F.col("movie_id").cast(StringType())).withColumn('production_countries',F.from_json(F.col('production_countries'),ArrayType(struct_countries)))

countries_exp = countries_sch.select('movie_id', F.explode_outer('production_countries').alias('production_countries'))
countries_cols = ["iso_3166_1", "name"]
for col in countries_cols:
    countries_exp = countries_exp.withColumn(col, F.col("production_countries").getItem(col))

countries = countries_exp.drop("production_countries").na.fill(value="XX", subset=["iso_3166_1"])

In [210]:
# countries.sort("id",ascending=False).show(truncate=False)

+--------+----------+------------------------+
|movie_id|iso_3166_1|name                    |
+--------+----------+------------------------+
|417859  |US        |United States of America|
|330770  |FR        |France                  |
|330770  |BE        |Belgium                 |
|330770  |ES        |Spain                   |
|324668  |US        |United States of America|
|324668  |CN        |China                   |
|324668  |GB        |United Kingdom          |
|315011  |JP        |Japan                   |
|308531  |HK        |Hong Kong               |
|308531  |US        |United States of America|
|308531  |CN        |China                   |
|297761  |US        |United States of America|
|293644  |IN        |India                   |
|293644  |MX        |Mexico                  |
|291805  |CN        |China                   |
|291805  |GB        |United Kingdom          |
|291805  |CA        |Canada                  |
|291805  |US        |United States of America|
|290595  |US 

# 2.8. spoken_languages

In [214]:
movie_languages = movies.withColumnRenamed("id", "movie_id").select("movie_id", "spoken_languages")
struct_languages = StructType([StructField('iso_639_1', StringType(), True), StructField('name', StringType(), True)])
languages_sch = movie_languages.withColumn('movie_id', F.col("movie_id").cast(StringType())).withColumn('spoken_languages',F.from_json(F.col('spoken_languages'),ArrayType(struct_languages)))

languages_exp = languages_sch.select('movie_id', F.explode_outer('spoken_languages').alias('spoken_languages'))
languages_cols = ["iso_639_1", "name"]
for col in languages_cols:
    languages_exp = languages_exp.withColumn(col, F.col("spoken_languages").getItem(col))

languages = languages_exp.drop("spoken_languages").na.fill(value="XX", subset=["iso_639_1"])

In [215]:
languages.sort("id",ascending=False).show(truncate=False)

+--------+---------+--------+
|movie_id|iso_639_1|name    |
+--------+---------+--------+
|417859  |es       |Español |
|417859  |en       |English |
|330770  |fr       |Français|
|324668  |en       |English |
|315011  |it       |Italiano|
|315011  |de       |Deutsch |
|315011  |en       |English |
|315011  |ja       |日本語  |
|308531  |en       |English |
|297761  |en       |English |
|293644  |XX       |null    |
|291805  |en       |English |
|291805  |zh       |普通话  |
|290595  |en       |English |
|286217  |en       |English |
|286217  |zh       |普通话  |
|281957  |fr       |Français|
|281957  |en       |English |
|278927  |en       |English |
|274854  |en       |English |
+--------+---------+--------+
only showing top 20 rows



# Check nulls NA from silver

In [216]:
spark.stop()